## Day 2: Password Philosophy 

Your flight departs in a few days from the coastal airport; the easiest way down to the coast from here is via toboggan.

The shopkeeper at the North Pole Toboggan Rental Shop is having a bad day. "Something's wrong with our computers; we can't log in!" You ask if you can take a look.

Their password database seems to be a little corrupted: some of the passwords wouldn't have been allowed by the Official Toboggan Corporate Policy that was in effect when they were chosen.

To try to debug the problem, they have created a list (your puzzle input) of passwords (according to the corrupted database) and the corporate policy when that password was set.

For example, suppose you have the following list:

```
1-3 a: abcde
1-3 b: cdefg
2-9 c: ccccccccc
```

Each line gives the password policy and then the password. The password policy indicates the lowest and highest number of times a given letter must appear for the password to be valid. For example, 1-3 a means that the password must contain a at least 1 time and at most 3 times.

In the above example, 2 passwords are valid. The middle password, cdefg, is not; it contains no instances of b, but needs at least 1. The first and third passwords are valid: they contain one a or nine c, both within the limits of their respective policies.

# Setup

In [46]:
using DataFrames

In [127]:
x = readlines("input-2.txt")
data = DataFrame(:x => x)
head(data)

,x
,String
1,2-4 p: vpkpp
2,6-16 b: bbbbbbbbbbbbbbbpb
3,6-7 z: zzfzzdz
4,4-6 q: tfzqvqcpcmqqjqzd
5,7-8 k: rkkkknkw
6,5-14 t: ttttnttttttdttttttt


In [134]:
data.min_val = parse.(Int, map(x -> x.match, match.(r"^\d{1,3}", data.x)))
data.max_val = parse.(Int, replace.(map(x -> x.match, match.(r"-\d{1,3}", data.x)), "-" => ""))
data.letter = map(x -> x.match, match.(r"[a-z]{1}", data.x))
data.pattern = map(x -> x.match, match.(r"[a-z]{2,}", data.x))
data.lengths = [length(collect(eachmatch(Regex(data.letter[i]), data.pattern[i]))) for i in 1:nrow(data)]
data.flag = (data.lengths .>= data.min_val) .& (data.lengths .<= data.max_val)
sum(data.flag)

474

In [136]:
data[data.flag .== 0,[:flag, :min_val, :max_val, :pattern, :lengths]]

,flag,min_val,max_val,pattern,lengths
,Bool,Int64,Int64,SubStri…,Int64
1,0,6,7,zzfzzdz,5
2,0,7,8,rkkkknkw,5
3,0,5,14,ttttnttttttdttttttt,17
4,0,3,4,hrht,2
5,0,2,6,ccccccc,7
6,0,5,7,pmtgqgg,3
7,0,16,18,vhhhhhhhhhhhhphhrnh,15
8,0,6,8,lllnllxb,5
9,0,3,4,zjwz,2


## Part Two 

While it appears you validated the passwords correctly, they don't seem to be what the Official Toboggan Corporate Authentication System is expecting.

The shopkeeper suddenly realizes that he just accidentally explained the password policy rules from his old job at the sled rental place down the street! The Official Toboggan Corporate Policy actually works a little differently.

Each policy actually describes two positions in the password, where 1 means the first character, 2 means the second character, and so on. (Be careful; Toboggan Corporate Policies have no concept of "index zero"!) Exactly one of these positions must contain the given letter. Other occurrences of the letter are irrelevant for the purposes of policy enforcement.

Given the same example list from above:

```
1-3 a: abcde is valid: position 1 contains a and position 3 does not.
1-3 b: cdefg is invalid: neither position 1 nor position 3 contains b.
2-9 c: ccccccccc is invalid: both position 2 and position 9 contain c.
```

How many passwords are valid according to the new interpretation of the policies?

In [149]:
data.first_pos = SubString.(data.pattern, data.min_val, data.min_val)
data.last_pos = SubString.(data.pattern, data.max_val, data.max_val)
data[:, [:letter, :first_pos, :last_pos]]

data.flag_pos = (data.first_pos .== data.letter) .+ (data.last_pos .== data.letter) 

sum(data.flag_pos .== 1)

745